In [2]:
import os
import re
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_ANCHOR

from pptx.util import Inches, Pt, Mm, Cm
from pptx.oxml.xmlchemy import OxmlElement
import cv2

from concurrent.futures import ThreadPoolExecutor
import numpy as np

import matplotlib.pyplot as plt

import pytesseract

def img_crop(image_path, crop_rect):
    with Image.open(image_path) as image:
        cropped_image = image.crop(crop_rect)
        cropped_image = ImageOps.grayscale(cropped_image)
        cropped_image = cropped_image.filter(ImageFilter.SHARPEN)
        
        enhancer = ImageEnhance.Contrast(cropped_image)
        cropped_image = enhancer.enhance(2.0)
        
        open_cv_image = np.array(cropped_image)
        
        _, binary = cv2.threshold(open_cv_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        kernel = np.ones((1, 1), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        binary = Image.fromarray(binary)
        
        cst_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789.'
        ocr_result = pytesseract.image_to_string(binary, lang='eng', config=cst_config)
        
        filter_result = ''.join([c for c in ocr_result if c.isdigit() or c == '.'])
        
        if not re.match(r'^\d*\.\d+$', filter_result):
            filter_result = re.sub(r'(\d{1,2})(\d{2})$', r'\1.\2', filter_result)
        
        return filter_result.strip()

def perform_ocr(image_path):
    crop_rect = [
        (436, 435, 502, 456),
        (437, 455, 503, 477),
        (68, 31, 151, 47)
    ]
    
    with ThreadPoolExecutor() as executor:
        tasks = [executor.submit(img_crop, image_path, rect) for rect in crop_rect]
        extracted_texts = [task.result() for task in tasks]

    return extracted_texts

def extract_number(text):
    match = re.search(r'\d+\.\d+', text)
    return match.group() if match else '0'

def set_table_border(table):
    for row in table.rows:
        for cell in row.cells:
            tc = cell._tc
            tcPr = tc.get_or_add_tcPr()

            for border_tag in ['a:lnL', 'a:lnR', 'a:lnT', 'a:lnB']:
                ln = OxmlElement(border_tag)
                ln.set('w', "8700")
                solidFill = OxmlElement('a:solidFill')
                srgbClr = OxmlElement('a:srgbClr')
                srgbClr.set('val', "BFBFBF")
                solidFill.append(srgbClr)
                ln.append(solidFill)
                tcPr.append(ln)

def new_line(cell, text):

    modified_text = re.sub(r'\s*(\[.*?\])', r'\n\1', text)
    
    cell.text = ''
    tf = cell.text_frame
    tf.vertical_anchor = MSO_ANCHOR.MIDDLE
    
    parts = modified_text.split('\n')
    for idx, part in enumerate(parts):

        if idx == 0:
            p = tf.paragraphs[0]
        else:
            p = tf.add_paragraph()

        p.text = part.strip()
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = "현대하모니 M"

        if idx != 0:
            p.font.color.rgb = RGBColor(191, 191, 191)

def add_table_slide(prs, values=None):
    
    prs.slide_width = Cm(27.517)
    prs.slide_height = Cm(19.05)

    slide_table = prs.slides.add_slide(prs.slide_layouts[0])
    table = slide_table.shapes.add_table(14, 14, Cm(0.76), Cm(2.97), Cm(25.94), Cm(15.89)).table

    txBox = slide_table.shapes.add_textbox(Cm(0.64), Cm(0.23), Cm(25), Cm(1.5))
    tf = txBox.text_frame

    p = tf.paragraphs[0]
    p.text = "▣ 노이즈 레벨 점검시트"
    p.font.bold = False
    p.font.size = Pt(24)
    p.font.name = "현대하모니 M"

    txBox_2 = slide_table.shapes.add_textbox(Cm(0.63), Cm(1.9), Cm(25), Cm(1.5))
    tf_2 = txBox_2.text_frame
        
    p_2 = tf_2.paragraphs[0]
    p_2.text = "▶ 노이즈 점검결과"
    p_2.font.bold = False
    p_2.font.size = Pt(14)
    p_2.font.name = "현대하모니 M"


    merge_cells = [
        ((0, 0), (1, 1)), ((2, 0), (5, 1)), ((6, 0), (9, 1)), ((10, 0), (13, 1)), ((0, 2), (0, 7)),
        ((1, 2), (1, 4)), ((1, 5), (1, 7)), ((2, 2), (5, 4)), ((6, 2), (9, 4)), ((10, 2), (13, 4)),
        ((2, 5), (5, 7)), ((6, 5), (9, 7)), ((10, 5), (13, 7)), ((0, 8), (1, 12)), ((0, 13), (1, 13)),
        ((2, 9), (2, 10)), ((2, 11), (2, 12)), ((3, 9), (3, 10)), ((3, 11), (3, 12)), ((4, 9), (4, 10)),
        ((4, 11), (4, 12)), ((5, 9), (5, 10)), ((5, 11), (5, 12)), ((6, 9), (6, 10)), ((6, 11), (6, 12)),
        ((7, 9), (7, 10)), ((7, 11), (7, 12)), ((8, 9), (8, 10)), ((8, 11), (8, 12)), ((9, 9), (9, 10)),
        ((9, 11), (9, 12)), ((10, 9), (10, 10)), ((10, 11), (10, 12)), ((11, 9), (11, 10)), ((11, 11), (11, 12)),
        ((12, 9), (12, 10)), ((12, 11), (12, 12)), ((13, 9), (13, 10)), ((13, 11), (13, 12)), ((2, 13), (5, 13)),
        ((6, 13), (9, 13)), ((10, 13), (13, 13))
    ]
    for merge_range in merge_cells:
        table.cell(*merge_range[0]).merge(table.cell(*merge_range[1]))
    
    widths_cm = [1.6, 1.6, 2.13, 2.13, 2.13, 2.13, 2.13, 2.13, 2.04, 1.525, 1.525, 1.525, 1.525, 1.8]

    for i, width_cm in enumerate(widths_cm):
        table.columns[i].width = Cm(width_cm)
        
    set_table_border(table)
    
    for i in range(14):
        for j in range(14):
            cell = table.cell(i, j)
            cell.fill.solid()
            cell.fill.fore_color.rgb = RGBColor(255, 255, 255) 
    data = [
        ((0, 0), "대상"), ((0, 2), "점검결과"), ((1, 2), "평균치 Av"), ((1, 5), "준 첨두치 Qp"), ((0, 8), "측정값 (㏈㎶)"), ((0, 13), "판정")
    ]


    table.rows[0].height =  Pt(15.346)
    table.rows[1].height =  Pt(15.346)

    cell_margin = Cm(0.1)

    for i, row in enumerate(table.rows):
        if i in [0, 1]:
            for cell in row.cells:
                cell.margin_top = cell_margin
                cell.margin_bottom = cell_margin

    for i in range(2, 14):
        table.rows[i].height = Cm(1.19)

    for (row, col), text in data:
        cell = table.cell(row, col)
        cell.text = text

        cell.text_frame.paragraphs[0].font.size = Pt(14)
        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(242, 242, 242)

        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.color.rgb = RGBColor(0, 0, 0)
                run.font.name = "현대하모니 M"
                run.font.bold = False
                run.font.italic = False
                run.font.underline = False
                
        cell.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

        for paragraph in cell.text_frame.paragraphs:
            paragraph.alignment = PP_ALIGN.CENTER
            paragraph.vertical_anchor = MSO_ANCHOR.MIDDLE

        cell.vertical_anchor = MSO_ANCHOR.MIDDLE
                    
    data = [
        ((2, 8), "구분"), ((2, 9), "0.15~0.5 ㎒ [66 ㏈㎶↓]"), ((2, 11), "0.5~5.0 ㎒ [60 ㏈㎶↓]"), ((3, 8), "평균치"),
        ((4, 8), "구분"), ((4, 9), "0.15~0.5 ㎒ [79 ㏈㎶↓]"), ((4, 11), "0.5~5.0 ㎒ [73 ㏈㎶↓]"), ((5, 8), "준 첨두치"),
        ((6, 8), "구분"), ((6, 9), "0.15~0.5 ㎒ [66 ㏈㎶↓]"), ((6, 11), "0.5~5.0 ㎒ [60 ㏈㎶↓]"), ((7, 8), "평균치"),
        ((8, 8), "구분"), ((8, 9), "0.15~0.5 ㎒ [79 ㏈㎶↓]"), ((8, 11), "0.5~5.0 ㎒ [73 ㏈㎶↓]"), ((9, 8), "준 첨두치"),
        ((10, 8), "구분"), ((10, 9), "0.15~0.5 ㎒ [66 ㏈㎶↓]"), ((10, 11), "0.5~5.0 ㎒ [60 ㏈㎶↓]"), ((11, 8), "평균치"),
        ((12, 8), "구분"), ((12, 9), "0.15~0.5 ㎒ [79 ㏈㎶↓]"), ((12, 11), "0.5~5.0 ㎒ [73 ㏈㎶↓]"), ((13, 8), "준 첨두치")
    ]
    for (row, col), text in data:
        cell = table.cell(row, col)
        new_line(cell, text)
        cell.text_frame.paragraphs[0].font.size = Pt(11)
        cell.text_frame.paragraphs[0].font.name = "현대하모니 M"
        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        cell.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(242, 242, 242)
        cell.vertical_anchor = MSO_ANCHOR.MIDDLE
        
    if values is not None:
        for (row, col), text in values:
            cell = table.cell(row, col)
            new_line(cell, text)
            cell.text_frame.paragraphs[0].font.size = Pt(10)
            cell.text_frame.paragraphs[0].font.name = "현대하모니 M"
            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            cell.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
            cell.vertical_anchor = MSO_ANCHOR.MIDDLE

    set_table_border(table)
    
def set_score(avg_left, avg_right, tmp_left, tmp_right):
    avg_criteria = [66, 60]
    tmp_criteria = [79, 73]
    
    
    if (avg_left > avg_criteria[0] and avg_right > avg_criteria[1] and
        tmp_left > tmp_criteria[0] and tmp_right > tmp_criteria[1]):
        return "T"
    else:
        return "F"





def process_image_names(image_files, match):
    if match == 'default':
        image_names = ['-'.join([os.path.splitext(filename)[0].split()[0], os.path.splitext(filename)[0].split()[1].split('-')[1]]) for filename in image_files]
    else:
        image_names = []
        for filename in image_files:
            parts = os.path.splitext(filename)[0].split()
            number_part = parts[0]
            line_part = parts[1]
            combined_name = f"{number_part} {line_part}"
            image_names.append(combined_name)
    
    return list(set(image_names))

def sort_image_names(image_names, match):
    if match == 'default':
        sorted_image_names = sorted(image_names, key=lambda x: int(re.search(r'(?<=-)\d+', x).group()))
    else:
        sorted_image_names = sorted(image_names, key=lambda x: (int(re.search(r'\d+', x).group()), re.search(r'[a-zA-Z]+', x).group()))
    
    return sorted_image_names

def generate_pptx(image_folder, output_path, match):
    image_files = [filename for filename in os.listdir(image_folder) if filename.endswith('.png')]
    
    image_names = process_image_names(image_files, match)
    tmp_names = list(set(image_names))
    g_i_files = [image_files[i:i+6] for i in range(0, len(image_files), 6)]
    
    print(tmp_names)

    ocr_results = []
    for group in g_i_files:
        ocr_group_results = []
        for image_path in group:
            ocr_result = perform_ocr(os.path.join(image_folder, image_path))
            print(ocr_result)
            ocr_group_results.append(ocr_result)
        ocr_results.append(ocr_group_results)
    
    f_numbers = [[extract_number(sublist[0]) for sublist in group_result] for group_result in ocr_results]
    s_numbers = [[extract_number(sublist[1]) for sublist in group_result] for group_result in ocr_results]

    sorted_image_names = sort_image_names(tmp_names, match)
    grouped_image_names = [sorted_image_names[i:i+3] for i in range(0, len(sorted_image_names), 3)]
    
    prs = Presentation()
    
    height = Inches(8.27) 
    width = Inches(11.69) 
    prs.slide_width = width
    prs.slide_height = height

    for i, group_name in enumerate(grouped_image_names):
        values = []
        for j, image_name in enumerate(group_name):
            value_index = j * 6
            f_len = len(f_numbers[i])
            s_len = len(s_numbers[i])
            for k in range(min(6, f_len - value_index)):
                f_value = f_numbers[i][k]
                s_value = s_numbers[i][k]

                values.append(((3 + k*2, 9), f_value))
                values.append(((3 + k*2, 11), s_value))
            values.append(((2 + j*4, 0), image_name))
            
            avg_left = float(f_numbers[i][j*2]) if j*2 < len(f_numbers[i]) else 0
            avg_right = float(s_numbers[i][j*2]) if j*2 < len(s_numbers[i]) else 0
            tmp_left = float(f_numbers[i][j*2+1]) if j*2+1 < len(f_numbers[i]) else 0
            tmp_right = float(s_numbers[i][j*2+1]) if j*2+1 < len(s_numbers[i]) else 0

            score = set_score(avg_left, avg_right, tmp_left, tmp_right)
            values.append(((2 + j*4, 13), score))
            
        add_table_slide(prs, values)

    prs.save(output_path)

generate_pptx('D:\\개발\\python\\Graph_PPT\\DX challenge\\img', 'D:\\개발\\python\\Graph_PPT\\DX challenge\\sdr.pptx', 'default')

IndexError: list index out of range

In [3]:
import os
from pptx import Presentation
from pptx.util import Cm

def add_images_to_existing_ppt(image_folder, existing_ppt):
    prs = Presentation(existing_ppt)
    slides_per_page = 6
    images = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))])
    total_images = len(images)

    slide_index = 0
    slide = prs.slides[slide_index]
    positions = [(Cm(4.06), Cm(4.62)), (Cm(10.46), Cm(4.62)),
                 (Cm(4.06), Cm(9.39)), (Cm(10.46), Cm(9.39)),
                 (Cm(4.06), Cm(14.16)), (Cm(10.46), Cm(14.16))]
    width = Cm(6.2)
    height = Cm(4.65)

    for i in range(total_images):
        idx_in_page = i % slides_per_page
        if idx_in_page == 0 and i != 0:
            slide_index += 1
            slide = prs.slides[slide_index]

        img_path = os.path.join(image_folder, images[i])
        left, top = positions[idx_in_page]
        slide.shapes.add_picture(img_path, left, top, width, height)

    prs.save(existing_ppt)

In [4]:
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

def browse_image_folder():
    folder_path = filedialog.askdirectory()
    img_folder_entry.delete(0, tk.END)
    img_folder_entry.insert(0, folder_path)

def browse_output_path():
    output_path = filedialog.asksaveasfilename(defaultextension=".pptx", filetypes=[("PowerPoint files", "*.pptx")])
    output_path_entry.delete(0, tk.END)
    output_path_entry.insert(0, output_path)

def generation():
    img_folder = img_folder_entry.get()
    output_path = output_path_entry.get()
    selected_format = format_var.get()

    if not img_folder or not output_path or not selected_format:
        messagebox.showerror("Error", "오류")
        return
    
    try:
        generate_pptx(img_folder, output_path, selected_format)
        add_images_to_existing_ppt(img_folder, output_path)
        messagebox.showinfo("Success", "성공")
    except Exception as e:
        messagebox.showerror("Error", f"오류 원인: {str(e)}")

root = tk.Tk()
root.title("서태정")

output_format_label = tk.Label(root, text="출력 형식 선택:")
output_format_label.grid(row=0, column=0, padx=10, pady=5, sticky="e")

format_var = tk.StringVar(root)
format_var.set("default")

output_format_menu = tk.OptionMenu(root, format_var, "default", "other")
output_format_menu.grid(row=0, column=1, padx=10, pady=5, sticky="w")

img_folder = tk.Label(root, text="폴더 선택:")
img_folder.grid(row=1, column=0, padx=10, pady=5, sticky="e")

img_folder_entry = tk.Entry(root, width=50)
img_folder_entry.grid(row=1, column=1, padx=10, pady=5)

img_folder_btn = tk.Button(root, text="Browse", command=browse_image_folder)
img_folder_btn.grid(row=1, column=2, padx=5, pady=5)

output_path = tk.Label(root, text="출력 경로:")
output_path.grid(row=2, column=0, padx=10, pady=5, sticky="e")

output_path_entry = tk.Entry(root, width=50)
output_path_entry.grid(row=2, column=1, padx=10, pady=5)

output_path_btn = tk.Button(root, text="Browse", command=browse_output_path)
output_path_btn.grid(row=2, column=2, padx=5, pady=5)

gen_btn = tk.Button(root, text="PPT 생성", command=generation)
gen_btn.grid(row=3, column=1, pady=10)

root.mainloop()


In [13]:
import tkinter as tk
from tkinter import filedialog, ttk

def browse_folder():
    folder_path = filedialog.askdirectory()
    if folder_path:
        folder_entry.delete(0, tk.END)
        folder_entry.insert(0, folder_path)

def browse_output():
    output_path = filedialog.asksaveasfilename(defaultextension=".pptx",
                                               filetypes=[("PowerPoint files", "*.pptx")])
    if output_path:
        output_entry.delete(0, tk.END)
        output_entry.insert(0, output_path)

def create_ppt():
    print("PPT 생성 버튼이 클릭되었습니다.")
    # 여기에 PPT 생성 코드를 추가할 수 있습니다.

# 메인 윈도우 설정
root = tk.Tk()
root.title("서태정")

# UI 요소들 배치
# 출력 형식 선택
format_label = tk.Label(root, text="출력 형식 선택:")
format_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

format_combobox = ttk.Combobox(root, values=["default", "option1", "option2"], state="readonly")
format_combobox.set("default")
format_combobox.grid(row=0, column=1, padx=10, pady=10, sticky="w")

# 폴더 선택
folder_label = tk.Label(root, text="폴더 선택:")
folder_label.grid(row=1, column=0, padx=10, pady=10, sticky="w")

folder_entry = tk.Entry(root, width=40)
folder_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")

folder_button = tk.Button(root, text="Browse", command=browse_folder)
folder_button.grid(row=1, column=2, padx=10, pady=10)

# 출력 경로
output_label = tk.Label(root, text="출력 경로:")
output_label.grid(row=2, column=0, padx=10, pady=10, sticky="w")

output_entry = tk.Entry(root, width=40)
output_entry.grid(row=2, column=1, padx=10, pady=10, sticky="w")

output_button = tk.Button(root, text="Browse", command=browse_output)
output_button.grid(row=2, column=2, padx=10, pady=10)

# PPT 생성 버튼
create_button = tk.Button(root, text="PPT 생성", command=create_ppt)
create_button.grid(row=3, column=1, pady=20)

# 윈도우 크기 조절 불가 설정
root.resizable(False, False)

# 메인 루프 시작
root.mainloop()


In [17]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QHBoxLayout, QWidget, QListWidget, QStackedWidget
from PyQt5.QtGui import QFont, QIcon
from PyQt5.QtCore import Qt

class MainUI(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Sidebar UI Example")
        self.setGeometry(100, 100, 800, 600)

        # Main layout
        main_layout = QHBoxLayout()

        # Sidebar
        sidebar = QListWidget(self)
        sidebar.setFixedWidth(200)
        sidebar.addItem("Dashboard")
        sidebar.addItem("Profile")
        sidebar.addItem("Messages")
        sidebar.addItem("Notifications")
        sidebar.addItem("Settings")
        sidebar.addItem("Sign Out")
        sidebar.currentRowChanged.connect(self.display_page)

        # Content area
        self.pages = QStackedWidget(self)
        self.pages.addWidget(self.create_page("Dashboard Page"))
        self.pages.addWidget(self.create_page("Profile Page"))
        self.pages.addWidget(self.create_page("Messages Page"))
        self.pages.addWidget(self.create_page("Notifications Page"))
        self.pages.addWidget(self.create_page("Settings Page"))
        self.pages.addWidget(self.create_page("Sign Out Page"))

        main_layout.addWidget(sidebar)
        main_layout.addWidget(self.pages)

        # Central widget
        central_widget = QWidget(self)
        central_widget.setLayout(main_layout)
        self.setCentralWidget(central_widget)

        # Style the interface
        self.setStyleSheet("""
            QMainWindow {
                background-color: #f4f4f4;
            }
            QListWidget {
                background-color: #007BFF;
                color: white;
                font-size: 18px;
                border: none;
                padding: 10px;
            }
            QListWidget::item {
                padding: 10px;
            }
            QListWidget::item:hover {
                background-color: #0056b3;
            }
            QListWidget::item:selected {
                background-color: #0056b3;
                border-left: 5px solid #ffffff;
            }
            QLabel {
                font-size: 24px;
                color: #333333;
            }
            QStackedWidget {
                background-color: #ffffff;
                border-radius: 10px;
                margin: 10px;
            }
        """)

    def create_page(self, text):
        page = QWidget()
        layout = QVBoxLayout()
        label = QLabel(text, self)
        label.setAlignment(Qt.AlignCenter)
        label.setFont(QFont('Arial', 24))
        layout.addWidget(label)
        page.setLayout(layout)
        return page

    def display_page(self, index):
        self.pages.setCurrentIndex(index)

def main():
    app = QApplication(sys.argv)
    window = MainUI()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


SystemExit: 0